In [1]:
import json
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer

from ast import literal_eval
from unidecode import unidecode

In [2]:
db_file = open('resume_base.json', 'rb')
db = json.loads(db_file.read())
db[0:2]

[{'id': 1,
  'title': 'Candidato de Contabilidade',
  'age': 49,
  'location': 'Centro, Curitiba - PR',
  'professional_goals_salary': '{"max": 0.0, "min": 0.0}',
  'professional_goals_contract_type': None,
  'professional_goals_working_hours': 'Período Integral',
  'education': '[{"dateEnd": "2003-11-01", "dateInit": "2003-02-01", "courseName": "Curso extra-curricular / Profissionalizante  em Contabilidade para executivos", "educationInstitutionName": "UNICENP em PR"}, {"dateEnd": "1989-12-01", "dateInit": "1985-02-01", "courseName": "Ensino Superior em Ciências contábeis", "educationInstitutionName": "FESP-Fundação de estudos Sociais do Parana em PR"}, {"dateEnd": "1984-11-01", "dateInit": "1982-02-01", "courseName": "Curso Técnico em Contabilidade", "educationInstitutionName": "CEPC-Centro de Estudos Profission.de Curitiba em PR"}]',
  'work_experience': '[{"field": "Contábil, Finanças, Economia", "level": null, "title": "CONTADOR", "dateEnd": "2006-12-01", "dateInit": "2005-03-01",

In [3]:
original_raw_data = pd.DataFrame(db)
raw_data = original_raw_data.copy()

In [4]:
[ s for s in raw_data['title'].unique() if 'mandrilagem' in s ]

['Candidato de mandrilagem']

In [5]:
from gensim.models import KeyedVectors

model = KeyedVectors.load_word2vec_format('cbow_s50.txt')
print('fitted')

C:\Users\mauri\Anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Slow version of gensim.models.doc2vec is being used


fitted


In [6]:
# sanitizing texts
# it had some characters not recognized by utf-code that needed to be manually changed

In [7]:
#for i in raw_data['title'].unique():
#    model.word_vec('museologia')

def is_clean(word):
    return word != '/' and len(word) > 2    
    
splitted_list = [ title.split(' ')[1:] for title in raw_data['title'].values ]
final_list = [ [ i.lower() for i in title if is_clean(i)] if ( '/' in title or len(title) > 1 ) else [title[0].lower()] for title in splitted_list ]

In [8]:
def clean_word(word):
    bad_characters = [ '(', ')', ',']
    for character in bad_characters:
        if character in word:
            word = word.replace(character, '')
    return word

for title in final_list:
    for word in title:
        test_word = clean_word(word)
        if test_word == 'aeropespacial':
            test_word = 'aeroespacial'
        elif test_word == 'serralheiria':
            test_word = 'serralheira'
        elif test_word == 'mandrilagem':
            test_word = 'mecânico'
        elif test_word == 'colorimetrista':
            test_word = 'colorista'
        elif test_word == 'fresaria':
            test_word = 'fresa'
        elif test_word == 'traumatorpedia':
            test_word = 'trauma'
        elif test_word == 'neorologia':
            test_word = 'neurologia'
        elif test_word == 'censoriamento':
            test_word = 'sensoriamento'
        model.word_vec(test_word)

In [9]:
model.word_vec('administração')

array([ 0.182897,  0.309039, -0.210265,  0.021087, -0.025528,  0.149114,
       -0.239802, -0.334962,  0.142423,  0.224987,  0.015808, -0.213538,
        0.225556, -0.0303  , -0.319418,  0.442861,  0.011046, -0.10348 ,
       -0.581525,  0.343898, -0.795083,  0.244718, -0.392894,  0.266937,
       -0.188591,  0.217794,  0.086318, -0.038021,  0.063617,  0.125227,
        0.267519, -0.382494,  0.118173, -0.46606 ,  0.447428,  0.310414,
       -0.489577, -0.204405,  0.415578, -0.028567,  0.284658,  0.339385,
       -0.045198,  0.124388, -0.506851, -0.19136 , -0.156576,  0.046356,
       -0.237123, -0.115663], dtype=float32)

In [10]:
df_test = raw_data.copy()
df_test['count'] = 1
df_test['count'] = df_test['count'].astype(int)
freq = df_test.groupby(by='title').sum()
freq.sort_values('count', ascending=False)

,age,id,count
title,,,
Candidato sem experiência incluida em CV,370500,527978275,18408
Candidato de Administração Geral,230240,461706384,9884
Candidato de Atendimento,122021,266344153,5205
Candidato de Produção,78996,192406469,3150
Candidato de Lojas / Shopping,74042,165897916,3140
Candidato de Venda Interna,71341,162644970,2810
Candidato de Recepção,62751,137199573,2725
Candidato de Telemarketing / Call Center Receptivo,46457,105829359,1892
Candidato de Venda Externa,42202,91270964,1563
